In [1]:
import numpy as np
import pandas as pd
import psycopg2
import getpass
# for configuring connection 
from configobj import ConfigObj
import os


In [ ]:
# Create a database connection using settings from config file
config='../db/config.ini'

# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = ''
    conn_info["sqlhost"] = '192.168.60.144'
    conn_info["sqlport"] = 6432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu_crd'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == '192.168.60.144') & (conn_info["sqlport"]=='6432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres


In [3]:
from sqlalchemy import create_engine,text
con= create_engine('postgresql://eicu@192.168.60.144:6432/eicu')

In [4]:
datadir = '/home/mei/nas/docker/dataset/EICU/eicu-collaborative-research-database-2.0/'
csv = '/home/mei/nas/docker/thesis/data/csv'

## flat_feature table: 
- select patient who comes from direct and emergency and the stay length in icu between 1 and 14 days

In [8]:
create_table_query =  query_schema +"""

drop materialized view if exists flat cascade;
create materialized view flat as
SELECT DISTINCT patientunitstayid, extract(hour from to_timestamp(unitadmittime24,'HH24:MI:SS')) as hour, gender, age, apacheadmissiondx, admissionweight,dischargeweight, unitdischargelocation,unitdischargestatus
FROM patient
WHERE unitadmitsource IN ('Emergency Department', 'Direct Admit') 
    AND unitdischargeoffset >=1440 
    AND unitdischargeoffset <=20160
group by patientunitstayid, hour
order by patientunitstayid, hour
"""

In [9]:
with con.begin() as connection:
    connection.execute(text(create_table_query))
select_query = "SELECT * FROM flat;"
df_flat = pd.read_sql_query(select_query, con)

In [10]:
df_flat

,patientunitstayid,hour,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
0,141168,15.0,Female,70,"Rhythm disturbance (atrial, supraventricular)",84.30,85.8,Death,Expired
1,141265,14.0,Male,67,"CVA, cerebrovascular accident/stroke",100.00,91.8,Floor,Alive
2,141266,21.0,Male,73,"Sepsis, renal/UTI (including bladder)",120.40,112.9,Floor,Alive
3,141276,14.0,Female,59,"Arrest, respiratory (without cardiac arrest)",156.60,156.6,Home,Alive
4,141284,23.0,Male,63,Anemia,NaN,88.5,Floor,Alive
...,...,...,...,...,...,...,...,...,...
68441,3353147,17.0,Male,24,"Overdose, sedatives, hypnotics, antipsychotics...",153.40,151.3,Floor,Alive
68442,3353194,10.0,Female,51,Cardiac arrest (with or without respiratory ar...,63.05,65.8,Death,Expired
68443,3353213,7.0,Female,51,Coma/change in level of consciousness (for hep...,54.40,59.4,Home,Alive
68444,3353226,18.0,Female,79,"Effusions, pleural",58.40,55.1,Death,Expired


##  labels
- unitdischarge location, unitdiscahrges status, actualiculos

In [11]:
create_table_query =  query_schema +"""
drop materialized view if exists labels cascade;
create materialized view labels as
  -- select all the data we need from the apache predictions table, plus patient identifier and hospital identifier
  -- information because we only want to select one episode per patient (more on this later)
  with all_labels as (
    select p.uniquepid, p.patienthealthsystemstayid, apr.patientunitstayid, p.unitvisitnumber,
      apr.predictedhospitalmortality, apr.actualhospitalmortality, apr.predictediculos, apr.actualiculos
      from patient as p
      inner join apachepatientresult as apr
        on p.patientunitstayid = apr.patientunitstayid
      -- only use the most recent apache prediction model and exclude anyone who doesn't have at least 24 hours of data
      where apr.apacheversion = 'IVa' and apr.actualiculos >= 1
    )
  select al.patientunitstayid, al.predictedhospitalmortality, al.actualhospitalmortality,
    al.predictediculos, al.actualiculos
    from all_labels as al
    -- 'selection' is a table which will choose a random hospital stay (the lowest number is fine because the stays
    -- are randomly ordered). In the case of multiple ICU stays within that hospital admission, it will choose the
    -- first ICU stay that satisfies the 24 hours of data requirement. The rationale is that the model should be
    -- applied as soon as there is 24 hours of continuous data within the hospital. This query extracts 89143 stays.
    inner join (
      select p.uniquepid, p.patienthealthsystemstayid, min(p.unitvisitnumber) as unitvisitnumber
        from patient as p
        inner join (
          select uniquepid, min(patienthealthsystemstayid) as patienthealthsystemstayid
            from all_labels
            group by uniquepid
          ) as intermediate_selection
          on p.patienthealthsystemstayid = intermediate_selection.patienthealthsystemstayid
        group by p.uniquepid, p.patienthealthsystemstayid
      ) as selection
      on al.patienthealthsystemstayid = selection.patienthealthsystemstayid
      and al.unitvisitnumber = selection.unitvisitnumber;
    inner join flat as f on la.patientunitstayid = f.patientunitstayid;
"""

In [12]:
select_query = "SELECT * FROM labels;"
df_labels = pd.read_sql_query(select_query, con)

In [13]:
df_labels

,patientunitstayid,predictedhospitalmortality,actualhospitalmortality,predictediculos,actualiculos
0,141168,5.9098619924959253E-2,EXPIRED,3.091127,2.4972
1,141194,0.1022830270275107,ALIVE,4.167129,3.3423
2,141203,0.4709733472175876,ALIVE,8.670299,1.2979
3,141227,0.48856191407785921,ALIVE,8.209624,1.1472
4,141233,3.5490436381315972E-2,ALIVE,3.107540,10.8923
...,...,...,...,...,...
89138,3353147,4.8307271745665827E-2,ALIVE,5.498694,1.0888
89139,3353190,-1,ALIVE,-1.000000,2.1187
89140,3353194,0.86952467986787907,EXPIRED,7.674351,2.4930
89141,3353213,5.2999142347007012E-2,ALIVE,5.801727,3.4118


In [14]:
flat_index = df_flat.set_index('patientunitstayid').index

In [15]:
df_labels = df_labels[df_labels['patientunitstayid'].isin(flat_index)]
col_labels = ['patientunitstayid','actualiculos']
col_flats = ['patientunitstayid','unitdischargelocation','unitdischargestatus']
df_labels = df_labels[col_labels].merge(df_flat[col_flats], left_on='patientunitstayid', right_on='patientunitstayid', how='inner')

In [16]:
df_labels

,patientunitstayid,actualiculos,unitdischargelocation,unitdischargestatus
0,141168,2.4972,Death,Expired
1,141265,4.2138,Floor,Alive
2,141266,1.0423,Floor,Alive
3,141276,1.1694,Home,Alive
4,141284,1.4416,Floor,Alive
...,...,...,...,...
51332,3353123,1.0409,Death,Expired
51333,3353140,1.3243,Skilled Nursing Facility,Alive
51334,3353147,1.0888,Floor,Alive
51335,3353194,2.4930,Death,Expired


## diagnosis

In [27]:
create_table_query =  query_schema +"""
drop materialized view if exists diagnoses cascade;
create materialized view diagnoses as

  -- for past medical history:
  select ph.patientunitstayid, ph.pasthistorypath as diagnosisstring
    from pasthistory as ph
    inner join labels as l on l.patientunitstayid = ph.patientunitstayid
    inner join flat as f on f.patientunitstayid = ph.patientunitstayid
    where ph.pasthistoryoffset < 1440
    and ph.pasthistorypath LIKE '%Organ Systems%'
       
"""

In [28]:
with con.begin() as connection:
    connection.execute(text(create_table_query)) 
    
select_query = "SELECT * FROM diagnoses;"
df_dx = pd.read_sql_query(select_query, con)

In [29]:
df_dx

,patientunitstayid,diagnosisstring
0,141168,notes/Progress Notes/Past History/Organ System...
1,141168,notes/Progress Notes/Past History/Organ System...
2,141168,notes/Progress Notes/Past History/Organ System...
3,141168,notes/Progress Notes/Past History/Organ System...
4,141168,notes/Progress Notes/Past History/Organ System...
...,...,...
170729,3353194,notes/Progress Notes/Past History/Organ System...
170730,3353194,notes/Progress Notes/Past History/Organ System...
170731,3353194,notes/Progress Notes/Past History/Organ System...
170732,3353194,notes/Progress Notes/Past History/Organ System...


## medication

In [5]:
query = query_schema + """
select a.patientunitstayid, a.drugname
from admissiondrug a
join flat f on a.patientunitstayid = f.patientunitstayid
  
order by a.patientunitstayid
"""

In [6]:
df_drug = pd.read_sql_query(query,con)

In [7]:
df_drug

,patientunitstayid,drugname
0,252784,LISINOPRIL ...
1,252784,ASPIRIN ...
2,252784,MULTIVITAMIN ...
3,252784,LANTUS ...
4,252784,NITROGLYCERIN ...
...,...,...
350668,3348105,METFORMIN HCL ...
350669,3348105,TRIAMTERENE W/HCTZ ...
350670,3348105,DILTIAZEM HCL ...
350671,3348105,DILTIAZEM HCL ...


## lab_ ts

In [30]:
create_table_query =  query_schema +"""
drop materialized view if exists timeserieslab cascade;
create materialized view timeserieslab as
  select l.patientunitstayid, l.labresultoffset, l.labname
    from lab as l
    inner join commonlabs as cl
      on cl.labname = l.labname  -- only include the common labs
    inner join labels as la
      on la.patientunitstayid = l.patientunitstayid -- only extract data for the cohort
    where l.labresultoffset between -1440 and 1440;

"""

In [31]:
select_query = "SELECT * FROM timeserieslab;"
df_timeserieslab = pd.read_sql_query(select_query, con)

In [32]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
0,141168,1133,PT - INR,2.50
1,141168,1133,PT,26.60
2,141168,231,PT - INR,1.70
3,141168,516,BUN,26.00
4,141168,231,PT,17.10
...,...,...,...,...
6814785,3353216,787,MCHC,32.00
6814786,3353216,787,MCH,30.00
6814787,3353216,787,Hct,26.00
6814788,3353216,787,RBC,2.79


## vital periodic

In [33]:
create_table_query =  query_schema +"""
drop materialized view if exists timeseriesperiodic cascade;
create materialized view timeseriesperiodic as
  select vp.patientunitstayid, vp.observationoffset, vp.temperature, vp.sao2, vp.heartrate, vp.respiration, vp.cvp,
    vp.systemicsystolic, vp.systemicdiastolic, vp.systemicmean, vp.st1, vp.st2, vp.st3
    from vitalperiodic as vp
    -- select only the patients who are in the cohort
    inner join labels as la
      on la.patientunitstayid = vp.patientunitstayid
    where vp.observationoffset between -1440 and 1440
    order by vp.patientunitstayid, vp.observationoffset;
"""

In [34]:
select_query = "SELECT * FROM timeseriesperiodic;"
df_timeseriesperiodic = pd.read_sql_query(select_query, con)

In [35]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
0,141168,119,NaN,93.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,124,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,129,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,134,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,139,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24337939,3353216,1417,NaN,95.0,85.0,25.0,NaN,166.0,105.0,120.0,0.0,0.0,0.0
24337940,3353216,1422,NaN,96.0,83.0,22.0,NaN,160.0,102.0,117.0,0.0,0.0,0.0
24337941,3353216,1427,NaN,96.0,86.0,23.0,NaN,163.0,101.0,116.0,0.0,0.0,0.0
24337942,3353216,1432,NaN,94.0,95.0,23.0,NaN,159.0,79.0,101.0,0.0,0.0,0.0


In [15]:
df_dx=pd.read_csv(csv+'/diagnoses.csv')
df_timeserieslab=pd.read_csv(csv+'/timeserieslab.csv')
df_timeseriesperiodic=pd.read_csv(csv+'/timeseriesperiodic.csv')
df_labels=pd.read_csv(csv+'/labels.csv')
df_flat=pd.read_csv(csv+'/flat.csv')


In [24]:
common_ids = set(df_labels['patientunitstayid'].unique()) \
             .intersection(df_dx['patientunitstayid'].unique()) \
             .intersection(df_timeserieslab['patientunitstayid'].unique()) \
             .intersection(df_timeseriesperiodic['patientunitstayid'].unique()) \
             .intersection(df_flat['patientunitstayid'].unique())\
             .intersection(df_drug['patientunitstayid'].unique())

In [25]:
# common_ids_df = pd.DataFrame(list(common_ids), columns=['patientunitstayid'])
# common_ids_df.to_csv(csv+'/common_ids.csv', index=False)

In [10]:
# common_ids=pd.read_csv(csv+'/common_ids.csv')

In [26]:
len(common_ids)

10496

In [27]:
def filter_and_sort(dataframes, common_ids, id_column='patientunitstayid'):
    """
    Filter and sort dataframes by common
    """
    return [
        df[df[id_column].isin(common_ids)].sort_values(by=id_column)
        for df in dataframes
    ]

In [28]:
dataframes = [df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat, df_drug]
filtered_sorted_dataframes = filter_and_sort(dataframes, common_ids)

# back to the original dataframes
df_labels, df_dx, df_timeserieslab, df_timeseriesperiodic, df_flat,df_drug = filtered_sorted_dataframes

In [29]:
def export_table_to_csv(df, output_file):
    """
    Export a SQL query to a CSV file
    """
    df.to_csv(output_file, index=False)
    print(f"Exported {output_file}")


In [31]:
## labels
export_table_to_csv(df_labels, f"{csv}/labels.csv")
## static data
export_table_to_csv(df_dx, f"{csv}/diagnoses.csv")
export_table_to_csv(df_flat, f"{csv}/flat.csv")
export_table_to_csv(df_drug, f"{csv}/drug.csv")

## time series data
export_table_to_csv(df_timeserieslab, f"{csv}/timeserieslab.csv")
export_table_to_csv(df_timeseriesperiodic, f"{csv}/timeseriesperiodic.csv")


Exported /home/mei/nas/docker/thesis/data/csv/labels.csv
Exported /home/mei/nas/docker/thesis/data/csv/diagnoses.csv
Exported /home/mei/nas/docker/thesis/data/csv/flat.csv
Exported /home/mei/nas/docker/thesis/data/csv/drug.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeserieslab.csv
Exported /home/mei/nas/docker/thesis/data/csv/timeseriesperiodic.csv


In [32]:
df_flat

,patientunitstayid,hour,gender,age,apacheadmissiondx,admissionweight,dischargeweight,unitdischargelocation,unitdischargestatus
2354,252784,15.0,Male,56,Diabetic ketoacidosis,75.00,75.7,Floor,Alive
2362,253331,1.0,Male,76,"CHF, congestive heart failure",114.70,114.4,Floor,Alive
2382,255112,5.0,Male,52,Cardiac arrest (with or without respiratory ar...,50.60,58.2,Death,Expired
2434,258354,19.0,Female,61,"Hypertension, uncontrolled (for cerebrovascula...",74.38,75.3,Floor,Alive
2452,259414,1.0,Male,81,"CVA, cerebrovascular accident/stroke",107.50,106.7,Floor,Alive
...,...,...,...,...,...,...,...,...,...
39538,3247116,21.0,Female,52,"Arrest, respiratory (without cardiac arrest)",61.70,68.0,Death,Expired
39543,3247360,21.0,Female,25,"Sepsis, renal/UTI (including bladder)",48.90,60.3,Floor,Alive
39544,3247421,1.0,Male,59,"Sepsis, pulmonary",76.20,82.1,Floor,Alive
40510,3346588,17.0,Female,71,"CHF, congestive heart failure",121.50,115.3,Telemetry,Alive


In [33]:
df_labels

,patientunitstayid,actualiculos,unitdischargelocation,unitdischargestatus
2354,252784,2.0500,Floor,Alive
2362,253331,1.7625,Floor,Alive
2382,255112,10.7381,Death,Expired
2434,258354,3.0090,Floor,Alive
2452,259414,5.8562,Floor,Alive
...,...,...,...,...
39538,3247116,8.1256,Death,Expired
39543,3247360,3.0097,Floor,Alive
39544,3247421,2.9520,Floor,Alive
40510,3346588,2.1826,Telemetry,Alive


In [34]:
df_dx

,patientunitstayid,diagnosisstring
7832,252784,notes/Progress Notes/Past History/Organ System...
7833,252784,notes/Progress Notes/Past History/Organ System...
7834,252784,notes/Progress Notes/Past History/Organ System...
7935,253331,notes/Progress Notes/Past History/Organ System...
7934,253331,notes/Progress Notes/Past History/Organ System...
...,...,...
167937,3347496,notes/Progress Notes/Past History/Organ System...
167936,3347496,notes/Progress Notes/Past History/Organ System...
167935,3347496,notes/Progress Notes/Past History/Organ System...
167941,3347496,notes/Progress Notes/Past History/Organ System...


In [35]:
df_drug

,patientunitstayid,drugname
0,252784,LISINOPRIL ...
10,252784,OMEPRAZOLE ...
9,252784,CARVEDILOL ...
8,252784,NOVOLOG ...
7,252784,OMEGA 3 ...
...,...,...
350623,3347496,CLOPIDOGREL BISULFATE ...
350625,3347496,XANAX ...
350626,3347496,LASIX ...
350628,3347496,FENOFIBRATE ...


In [36]:
df_timeserieslab

,patientunitstayid,labresultoffset,labname,labresult
162935,252784,1020,phosphate,2.0
163030,252784,1140,bedside glucose,97.0
163031,252784,1405,bedside glucose,170.0
163032,252784,20,paO2,1143.8
163033,252784,1349,bedside glucose,170.0
...,...,...,...,...
2896530,3347496,55,-eos,1.0
2896531,3347496,55,Hct,34.0
2896532,3347496,55,BUN,42.0
2896534,3347496,55,glucose,96.0


In [37]:
df_timeseriesperiodic

,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,systemicsystolic,systemicdiastolic,systemicmean,st1,st2,st3
658304,252784,965,NaN,99.0,85.0,15.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0
658499,252784,40,NaN,100.0,112.0,25.0,NaN,NaN,NaN,NaN,0.00,0.00,0.0
658498,252784,10,NaN,100.0,106.0,22.0,NaN,NaN,NaN,NaN,0.05,0.05,0.0
658497,252784,15,NaN,100.0,107.0,24.0,NaN,NaN,NaN,NaN,0.10,0.00,-0.1
658496,252784,20,NaN,100.0,109.0,21.0,NaN,NaN,NaN,NaN,0.00,0.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11186416,3347496,1392,NaN,NaN,86.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11186415,3347496,1387,NaN,NaN,88.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11186414,3347496,1382,NaN,NaN,86.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11186420,3347496,1412,NaN,NaN,81.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
